# Notebook 2.4.3: The Coffee Cooling Problem

---

<br>

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---

Often the actual laws of physics can be modeled as differential equations.   The behavior of thermal systems--systems where the temperature of objects change as heat transfers from one object to another--is one place where this is true.  So in this notebook we'll do some physics!  

<br>

Our problem is a classic modeling problem involving a cup of coffee.  Suppose you stop on the way to school to pick up a cup of coffee and a small container of milk. Assuming that you want the coffee to be as hot as possible when you arrive at work, should you add the milk at the coffee shop, wait until you get to school, or add the milk at some point in between?

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/2_4/wired.PNG width = 400>
</center>

## Investigating the System

We'll follow the six steps in modeling over the next couple of notebooks.  First: investigation!

### Part 1: Describing the system

To help model our problem, we need to some basic data about our system, and we need to understand something about thermal systems.  First, we'll gather some data to describe the system: you make a trial run with the milk and coffee in separate containers and take some measurements:

<br>

-   When served, the temperature of the coffee is 90 °C. The volume is
    300 mL.

-   The milk is at an initial temperature of 5 °C, and you take
    50 mL.

-   The ambient temperature in the car is 22 °C.

-   The coffee is served in a well insulated cup. Still, when you arrive at school after 30 minutes (after a stop for a donut), the temperature of the coffee has fallen to 70 °C.

-   The milk container is not as well insulated. After 15 minutes, it
    warms up to 20 °C, nearly the ambient temperature.




### Part 2: Understanding the physics

To understand how coffee cools (and milk warms), we need a little background information about thermal systems. *Temperature* is a property of an object or a
system; in SI units it is measured in degrees Celsius (°C). Temperature quantifies how hot or cold the object is, which is related to the average velocity of the molecular particles in the object.

<br>

When particles in a hot object contact particles in a cold object, the
hot object gets cooler and the cold object gets warmer as energy is
transferred from one to the other. The transferred energy is called
*heat*; in SI units it is measured in joules (J).  Heat is related to temperature by the following equation, assuming that the object is made from a single material:

<br>

$$Q = m~c_p~\Delta T$$

<br>

where $Q$ is the amount of heat transferred to an object, $m$ is the mass of the object, $\Delta T$ is resulting change in temperature, and $c_p$ is the *specific heat* of the material.  Specific heat describes how much heat energy is needed to raise the temperature of a particular material.



So the equation above tells us how heat energy and temperature are related.   But how does the heat transfer from one object to another?  There are three ways:

<br>

-   Conduction: When objects at different temperatures come into
    contact, the faster-moving particles of the higher-temperature
    object transfer kinetic energy to the slower-moving particles of the lower-temperature object.

-   Convection: When particles in a gas or liquid flow from place to
    place, they carry heat energy with them. Fluid flows can be caused
    by external action, like stirring, or by internal differences in
    temperature. For example, you might have heard that hot air rises,
    which is a form of "natural convection".

-   Radiation: As the particles in an object move due to thermal energy,
    they emit electromagnetic radiation. The energy carried by this
    radiation depends on the object's temperature and surface properties

<br>

You can start to see that there is a lot going on this system: a hot cup of coffee probably loses heat in all three of these ways, and the amount of loss might be affected by details of the liquids, the carrying containers, and the environments.  So we'll want to do some abstraction...



---

<br>

🟨 🟨  Active reading

What simplifications could we make to our model?  List at least 4 "abstractions" that seem reasonable to you.  Which of these seems most significant?  

✅ ✅ Answer here.

---

## Abstracting the System

### Part 1: Simplifying the system


We'll need to assign mass and specific heat for the coffee and milk.  The specific heat capacity of coffee is probably close to that
of water, which is 4.2 J/g/°C. Assuming that the density of coffee is
close to that of water, which is 1 g/mL, the mass of 300 mL of coffee is 300 g, and the thermal mass $m~c_p$ is 1260 J/°C.

<br>

So when a cup of coffee cools from 90 °C to 70 °C, the change in
temperature, $\Delta T$ is 20 °C, which means that approximately $1260 J/°C * 20°C \approx 25,000 J$ of heat
energy was transferred from the cup and the coffee to the surrounding environment (the cup holder and air in the car), which we'll assume stays at the same temperature throughout the drive.

<br>

To give you a sense of how much energy that is, if you were able to
harness all of that heat to do work (which you cannot), you could
use it to lift a cup of coffee from sea level to 8571 m, just shy of the height of Mount Everest, 8848 m.

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/2_4/hot.PNG width = 400>
</center>

<br>

We'll make one more simplification. If the cup has substantial thermal mass, like a ceramic mug, we might consider a model that computes the temperature of coffee and cup separately.  But we'll assume that the cup is made of paper.  In this case, the cup has much less mass than the coffee, and the specific heat capacity of paper is lower, too, so it won't store much heat. In this case, it is reasonable to treat the cup and coffee as a single object.

---

<br>

🟨 🟨  Active reading

What abstractions/ simplifications did we make in this section?  Mention at least 2.  Which of the two seems most likely to result in significant error?

✅ ✅ Answer here.

---

### Part 2: Simplifying the physics

We can do more than just simplifying the physical characteristics of the materials: we can simplify the physical laws that we will model.

<br>

For instance, for objects like coffee in a car, the effect of radiation is much
smaller than the effects of conduction and convection.  Likewise, convection can be a complex topic, since it often depends on details of fluid flow in three dimensions.

<br>

Instead of working out all the details of radiation and convection, we can use a simplified model called "Newton's law of cooling".  These model takes all the complexity of conduction, convection, and radiation and boils them down to a constant *total heat transfer coefficient* $r$.

This law of cooling asserts that the temperature rate of change for an object is proportional to the difference in temperature between the object and the surrounding environment:

<br>

$$\frac{dT}{dt} = -r (T - T_{env})$$

<br>

where $t$ is time, $T$ is the temperature of the object, and $T_{env}$ is the temperature of the environment.

You may have noticed that Newton's so-called "law " is really a model: it is a good approximation in some conditions and less good in others.  For example, if the primary mechanism of heat transfer is conduction,
Newton's law is "true", which is to say that $r$ is constant over a
wide range of temperatures. And sometimes we can estimate $r$ based on
the material properties and shape of the object.

<br>

When convection contributes a non-negligible fraction of heat transfer, $r$ depends on temperature, but Newton's law is often accurate enough, at least over a narrow range of temperatures. In this case $r$ usually has to be estimated experimentally, since it depends on details of surface shape, air flow, evaporation, etc.

<br>

When radiation makes up a substantial part of heat transfer, Newton's
law is not a good model at all. This is the case for objects in space or in a vacuum, and for objects at high temperatures (more than a few
hundred degrees Celsius, say).

<br>

However, for a situation like the coffee cooling problem, we expect Newton's model to be quite good.

---

<br>

🟨 🟨  Active reading: Matching  (Enter the letters in correct order, separated by a space)

In [ ]:
import pandas as pd
import numpy as np
from urllib.request import urlretrieve

location = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/'
folder = 'Support_files/'
name = 'Embedded_Qs.ipynb'
local, _ = urlretrieve(location + folder + name, name)
%run /content/$name

#@title { form-width: "50%", display-mode: "form" }
home = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Embedded_Qs/'
data, number = display_match('DEQ', home,6)
answer = "" #@param {type:"string"}
a = answer.split(sep=" ")
check_match(data,a, number)

Would Newton's Law of Cooling be a good model for each of these situations?

Terms

1) A frying pan cooling down to room temperature
2) A satelite heating up upon reentering the atmosphere
3) A pond of water cooling in the open air after being used to drive a steam turbine
4) A hot ember radiating heat to the night sky

Definitions

A) Yes, Newton's Law of Cooling would give decent results.
B) No, the simplifications of the Law of Cooling would create large errors.


---

<br>

🟨 🟨 Active Reading: Matching (Enter the letters in correct order, separated by a space)

In [ ]:
#@title { form-width: "50%", display-mode: "form" }
home = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Embedded_Qs/'
data, number = display_match('2_4_DEQ', home,7)
answer = "" #@param {type:"string"}
a = answer.split(sep=" ")
check_match(data,a, number)

We have the structure of our model laid out.  What type of variable is each one of these?

Terms

1) Mass of coffee
2) Temperature of the coffee during the drive
3) Temperature of the coffee at the end of the simulation
4) Specific heat of the coffee
5) Time that we add the milk to the coffee

Definitions

A) State variable
B) Independent variable
C) Metric
D) Parameter


---